In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
%aimport PlottingTools
%aimport uniprotBedTools

In [3]:
SFARI_variants = pd.read_csv('../data/iWES_v2.gatk.pvcf_variants.tsv', sep = '\t')
SFARI_variants

,chrom,pos,ref,alt,af
0,chr1,69081,G,C,0.000187
1,chr1,69130,C,G,9e-06
2,chr1,69134,A,G,4.2e-05
3,chr1,69135,A,G,5e-06
4,chr1,69149,T,A,9e-06
...,...,...,...,...,...
12443148,chrY,25044036,CTGG,"CTGAG,C","1.9e-05,5e-06"
12443149,chrY,25044040,T,A,1.9e-05
12443150,chrY,25044041,CT,C,1.9e-05
12443151,chrY,25044046,G,A,1.4e-05


In [4]:
SFARI_variants["alt"] = SFARI_variants["alt"].astype(str).str.split(",")
SFARI_variants["af"] = SFARI_variants["af"].astype(str).str.split(",")

In [5]:
expanded_SFARI_variants = SFARI_variants.explode(['alt', 'af'])
expanded_SFARI_variants

,chrom,pos,ref,alt,af
0,chr1,69081,G,C,0.000187
1,chr1,69130,C,G,9e-06
2,chr1,69134,A,G,4.2e-05
3,chr1,69135,A,G,5e-06
4,chr1,69149,T,A,9e-06
...,...,...,...,...,...
12443150,chrY,25044041,CT,C,1.9e-05
12443151,chrY,25044046,G,A,1.4e-05
12443152,chrY,25044050,ATGG,A,1.4e-05
12443152,chrY,25044050,ATGG,ATAG,5e-06


In [8]:
single_nt_ref = expanded_SFARI_variants[expanded_SFARI_variants["ref"].str.len() == 1]
single_nt_ref

,chrom,pos,ref,alt,af
0,chr1,69081,G,C,0.000187
1,chr1,69130,C,G,9e-06
2,chr1,69134,A,G,4.2e-05
3,chr1,69135,A,G,5e-06
4,chr1,69149,T,A,9e-06
...,...,...,...,...,...
12443145,chrY,25044016,G,A,9e-06
12443146,chrY,25044034,A,C,5e-06
12443147,chrY,25044035,T,C,1.9e-05
12443149,chrY,25044040,T,A,1.9e-05


In [9]:
single_nt_ref_alt = single_nt_ref[single_nt_ref["alt"].str.len() == 1]
single_nt_ref_alt

,chrom,pos,ref,alt,af
0,chr1,69081,G,C,0.000187
1,chr1,69130,C,G,9e-06
2,chr1,69134,A,G,4.2e-05
3,chr1,69135,A,G,5e-06
4,chr1,69149,T,A,9e-06
...,...,...,...,...,...
12443145,chrY,25044016,G,A,9e-06
12443146,chrY,25044034,A,C,5e-06
12443147,chrY,25044035,T,C,1.9e-05
12443149,chrY,25044040,T,A,1.9e-05


In [11]:
snps_only = single_nt_ref_alt
snps_only

,chrom,pos,ref,alt,af
0,chr1,69081,G,C,0.000187
1,chr1,69130,C,G,9e-06
2,chr1,69134,A,G,4.2e-05
3,chr1,69135,A,G,5e-06
4,chr1,69149,T,A,9e-06
...,...,...,...,...,...
12443145,chrY,25044016,G,A,9e-06
12443146,chrY,25044034,A,C,5e-06
12443147,chrY,25044035,T,C,1.9e-05
12443149,chrY,25044040,T,A,1.9e-05


In [47]:
SFARI_TFs = pd.read_csv("../data/SFARI_TFs.csv")
SFARI_TFs

,Unnamed: 0,status,gene-symbol,gene-name,ensembl-id,chromosome,genetic-category,gene-score,syndromic,eagle,number-of-reports,GeneName,ProteinSeq,uniprotID
0,0,9,ADNP,Activity-dependent neuroprotector homeobox,ENSG00000101126,20,"Rare Single Gene Mutation, Syndromic, Functional",1.0,1,41.50,64,sp|Q9H2P0|ADNP_HUMAN,MFQLPVNNLGSLRKARKTVKKILSDIGLEYCKEHIEDFKQFEPNDF...,Q9H2P0
1,1,9,AHDC1,AT-hook DNA binding motif containing 1,ENSG00000126705,1,"Rare Single Gene Mutation, Syndromic",1.0,1,14.25,24,sp|Q5TGY3|AHDC1_HUMAN,MRVKPQGLVVTSSAVCSSPDYLREPKYYPGGPPTPRPLLPTRPPAS...,Q5TGY3
2,2,9,ARID2,AT-rich interaction domain 2,ENSG00000189079,12,"Rare Single Gene Mutation, Syndromic",2.0,1,NaN,12,sp|Q68CP9|ARID2_HUMAN,MANSTGKAPPDERRKGLAFLDELRQFHHSRGSPFKKIPAVGGKELD...,Q68CP9
3,3,9,ARNT2,aryl-hydrocarbon receptor nuclear translocator 2,ENSG00000172379,15,"Rare Single Gene Mutation, Syndromic, Genetic ...",2.0,0,NaN,15,sp|Q9HBZ2|ARNT2_HUMAN,MATPAAVNPPEMASDIPGSVTLPVAPMAATGQVRMAGAMPARGGKR...,Q9HBZ2
4,4,9,ARX,aristaless related homeobox,ENSG00000004848,X,"Rare Single Gene Mutation, Syndromic",1.0,1,13.80,24,sp|Q96QS3|ARX_HUMAN,MSNQYQEEGCSERPECKSKSPTLLSSYCIDSILGRRSPCKMRLLGA...,Q96QS3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,122,9,ZNF711,zinc finger protein 711,ENSG00000147180,X,Rare Single Gene Mutation,2.0,0,NaN,5,sp|Q9Y462|ZN711_HUMAN,MDSGGGSLGLHTPDSRMAHTMIMQDFVAGMAGTAHIDGDHIVVSVP...,Q9Y462
123,123,9,ZNF713,Zinc finger protein 713,ENSG00000178665,7,Rare Single Gene Mutation,2.0,0,NaN,3,sp|Q8N859|ZN713_HUMAN,MEEEEMNDGSQMVRSQESLTFQDVAVDFTREEWDQLYPAQKNLYRD...,Q8N859
124,124,9,ZNF774,Zinc finger protein 774,ENSG00000196391,15,Rare Single Gene Mutation,2.0,0,NaN,4,sp|Q6NX45|ZN774_HUMAN,MWLGTSGKSGLPGHCLENPLQECHPAQLEEWALKGISRPSVISQPE...,Q6NX45
125,125,9,ZNF804A,Zinc finger protein 804A,ENSG00000170396,2,"Rare Single Gene Mutation, Genetic Association...",2.0,0,NaN,16,sp|Q7Z570|Z804A_HUMAN,MECYYIVISSTHLSNGHFRNIKGVFRGPLSKNGNKTLDYAEKENTI...,Q7Z570


In [28]:
expanded_SFARI_variants["af"] = expanded_SFARI_variants["af"].astype(float)

In [34]:
np.percentile(expanded_SFARI_variants["af"], 95)

0.000337

In [35]:
top_5_percent = expanded_SFARI_variants[expanded_SFARI_variants["af"] >= 0.000337]

In [36]:
top_5_percent

,chrom,pos,ref,alt,af
12,chr1,69270,A,[G],0.033599
27,chr1,69428,T,[G],0.015504
42,chr1,69496,G,[A],0.000365
45,chr1,69511,A,[G],0.424581
54,chr1,69552,G,"[C, A]",0.000712
...,...,...,...,...,...
12442885,chrY,21548995,A,[G],0.000351
12442943,chrY,21904151,G,[A],0.003129
12442988,chrY,22169626,GAA,"[GA, G]",0.002736
12443087,chrY,23229711,T,[C],0.000422


In [40]:
top_5_percent.groupby(["chrom", "pos"]).count()

ref  alt  af
chrom pos                   
chr1  69270       1    1   1
      69428       1    1   1
      69496       1    1   1
      69511       1    1   1
      69552       1    1   1
...             ...  ...  ..
chrY  21548995    1    1   1
      21904151    1    1   1
      22169626    1    1   1
      23229711    1    1   1
      23229718    1    1   1

[659800 rows x 3 columns]

In [4]:
# bed files made in SFARI variants and TF intersection.ipynb

! bedtools intersect -a ../data/SFARI_variants.bed -b ../data/SFARI_TFs.bed > ../output/SFARI_variants_on_TFs.bed

In [44]:
pd.read_csv("../data/sorted_SFARI_variants.bed", sep = "\t", header = None)

,0,1,2
0,chrchr1,69080,69081
1,chrchr1,69129,69130
2,chrchr1,69133,69134
3,chrchr1,69134,69135
4,chrchr1,69148,69149
...,...,...,...
12443148,chrchrY,25044035,25044036
12443149,chrchrY,25044039,25044040
12443150,chrchrY,25044040,25044041
12443151,chrchrY,25044045,25044046
